In [ ]:
import re
import networkx as nx
from networkx.readwrite import json_graph
from fourlang.text_to_4lang import TextTo4lang
from fourlang.lexicon import Lexicon
from graphviz import Source
from scripts.parse_data import read
from scripts.check_output import compare_graphs
from tqdm import tqdm

In [ ]:
data_frame = read("de", graded=False)

In [ ]:
text_to_4lang_en = TextTo4lang(lang="de", port=5006)

In [ ]:
fourlang_votes = []
with open("result_binary", "r+") as f:
    for line in f:
        line = line.split()
        fourlang_votes.append(line[2])

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
text_to_4lang_en = TextTo4lang(lang="en")
text_to_4lang_de = TextTo4lang(lang="de", port=5006)

In [ ]:
graph_hypothesis = text_to_4lang_de.process_text("volleyball", True, 1, [])
print(text_to_4lang_de.get_definition("Pizza"))
dot_graph_premise = graph_hypothesis.to_dot()
Source(dot_graph_premise)

In [ ]:
def asim_jac_nodes(graph_premise, graph_hypothesis):
    prem = set(graph_premise.get_nodes())
    hyp = set(graph_hypothesis.get_nodes())
    prem = set([i.lower() for i in prem])
    hyp = set([i.lower() for i in hyp])
    sim = hyp & prem
    if not sim or len(hyp) == 0:
        return 0
    else:
        return float(len(sim)) / len(hyp)

In [ ]:
text_to_4lang_de.parser_wrapper.load_from_dict()

In [ ]:
text_to_4lang_de.get_definition("Dach")

In [ ]:
graph_hypothesis = text_to_4lang_de.process_text("dachgeschoss", True, 2, [], True)
dot_graph_premise = graph_hypothesis.to_dot()
Source(dot_graph_premise)

In [ ]:
graph_hypothesis = text_to_4lang_de.process_text("dachgeschoss", True, 1, [], False)

In [ ]:
text_to_4lang_de.parser_wrapper.parse["Arbeitslosigkeit"]

In [ ]:
preds = []
preds_graded = []
for i in range(len(data_frame)):
    index = i
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    score = data_frame.score[index]
    graph_premise = text_to_4lang_de.process_text(premise, True, 3, ["in", "auf"]) #legyen-e expand
    graph_hypothesis = text_to_4lang_de.process_text(hypothesis, True, 1, [], False)
    pred = asim_jac_nodes(graph_premise, graph_hypothesis)
    preds_graded.append(pred)
    if pred == 1.0:
        preds.append(1)
        if score == 0:
            print(premise + "\t" + hypothesis)
    else:
        preds.append(0)

In [ ]:
preds = []
preds_graded = []
for i in tqdm(range(len(data_frame))):
    index = i
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    premise = re.sub("_", " ", premise)
    hypothesis = re.sub("_", " ", hypothesis)
    score = data_frame.score[index]
    graph_premise = text_to_4lang_it.process_text(premise, True, 3, ["di", "su", "il"]) #legyen-e expand
    graph_hypothesis = text_to_4lang_it.process_text(hypothesis, True, 1)
    pred = asim_jac_nodes(graph_premise, graph_hypothesis)
    preds_graded.append(pred)
    if pred == 1.0:
        preds.append(1)
    else:
        preds.append(0)

In [ ]:
def get_4lang_score(premise, hypothesis):
    graph_premise = text_to_4lang_en.process_text(premise, True, 3) #legyen-e expand
    graph_hypothesis = text_to_4lang_en.process_text(hypothesis, True, 1)
    graph_premise.filter_graph("part")
    graph_premise.filter_graph("on")
    graph_premise.filter_graph("use")
    pred = asim_jac_nodes(graph_premise, graph_hypothesis)
    return pred

In [ ]:
preds = []
for j in tqdm(range(len(data_frame))):
    index = j
    premise = data_frame.premise[index]
    hypothesis = data_frame.hypothesis[index]
    score = data_frame.score[index]
    
    hyp_syn_names_all = []
    hyper_premise_names_all = []
    
    premise_syns = wn.synsets(premise, lang='ita')
    hyp_syns = wn.synsets(hypothesis, lang='ita')
    """
    if len(premise_syns) > 0 and len(hyp_syns) > 0:
        en_premise = premise_syns[0].lemmas()[0].name()
        en_hyp = hyp_syns[0].lemmas()[0].name()
        fourlang_score = get_4lang_score(en_premise, en_hyp)
    else:
        fourlang_score = 0
    """
    
    
    for premise_syn in premise_syns:

        hyperpremise = set([i for i in premise_syn.closure(lambda s:s.hypernyms())])

        hyper_premise_lemmas = []
        for i in hyperpremise:
            lemmas = i.lemmas()
            for lemm in lemmas:
                hyper_premise_lemmas.append(lemm)

        hyper_premise_names = set([i.name() for i in hyper_premise_lemmas])
        hyper_premise_names_all += list(hyper_premise_names)
        
    for hyp_syn in hyp_syns:
        hyp_syn_lemmas = hyp_syn.lemmas()
        hyp_syn_names = set([i.name() for i in hyp_syn_lemmas])
        
        hyp_syn_names_all += list(hyp_syn_names)
    
    if (set(hyp_syn_names_all) & set(hyper_premise_names_all)) or fourlang_votes[index] == '1':
        preds.append(1)
    else:
        if score == 1:
            print(premise + "\t" + hypothesis)
        preds.append(0)

In [ ]:
with open("result_binary", "w+") as f:
    for i,pred in enumerate(preds):
        premise = data_frame.premise[i]
        hypothesis = data_frame.hypothesis[i]
        f.write(premise + " " + hypothesis + " " + str(pred) + "\n")

In [ ]:
premise_syns = wn.synsets("servierer", lang='eng')
print(premise_syns)